In [ ]:
import os
import pickle
import pandas as pd
from pyAudioAnalysis import audioTrainTest as aT

audio_path = "data_audio"
all_files = []
labels = pd.read_csv("project_data/labels.csv", usecols=[0,1], dtype={"id": int, "label": str}, index_col=0)


def get_label_by_story(subject, story):
    return labels.loc[subject].values[0][story-1]

for root, dirs, files in os.walk(audio_path):
    for f in files:
        all_files.append(os.path.join(root, f))

predictions = {}
for trfile in all_files:
    originalname = trfile
    print originalname
    inter = originalname.split("/")[-1].strip("sub").strip(".wav").split("_")
    sub, st = int(inter[0]), int(inter[1])
    movedname = originalname.replace(originalname.split("/")[1], "fold")
    os.rename(originalname, movedname)
    aT.featureAndTrain(["data_audio/lie","data_audio/truth"], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "audio_models/svm/svm", True)
    prediction = aT.fileClassification(movedname, "audio_models/svm/svm", "svm")[1][1]
    # prediction = 1 if prediction>0.5 else 0
    predictions[(sub, st)] = [prediction, get_label_by_story(sub, st)]
    os.rename(movedname, originalname)

with open("audio_probability_prediction.pickle", "wb") as f:
    pickle.dump(predictions, f)

In [16]:
import pickle
def deserialize(f):
    with open(f, "rb") as ser:
        return pickle.load(ser)

In [18]:
import pandas as pd
label_folder = "project_data/labels.csv"
game = pd.read_csv(label_folder, usecols=[0, 3], dtype={"id": int, "game": int}, index_col=0)
game.loc[0].values[0]==0

True

In [37]:
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error as mse
audio_prediction = deserialize("audio_probibility_prediction.pickle")
results = np.array(audio_prediction.values()).astype(int)
print mse(results[:, 0], results[:, 1])
# results = results[:, 0] - results[:, 1]
# results = (len(audio_prediction) - sum(abs(results)))/float(len(audio_prediction))
# print "Correctly classification rate: {:.4f}%".format(results*100)

IOError: [Errno 2] No such file or directory: 'audio_probability_prediction.pickle'